In [1]:
#AIT 726-DL1
#Professor: Dr. Lindi Liao
#Project Baseline Solution
#Title: LLMs for Software Engineering -- Code Generation Using LLMs and Evaluation
#Project Team 5
#Mukesh Rajmohan - G01456275
#Utkarsh Jayesh Desai- G01419940
#Praneeth Ravirala - G01448129

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Importing Required Libraries
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, TensorDataset, RandomSampler
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW

In [ ]:
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))

True
0
Tesla T4


In [ ]:
#Importing the dataset
data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sql_code_dataset.csv')

In [ ]:
#Retrieving the structure of the dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78577 entries, 0 to 78576
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  78577 non-null  object
 1   context   78577 non-null  object
 2   answer    78577 non-null  object
dtypes: object(3)
memory usage: 1.8+ MB


In [ ]:
#Due to limited computing resources, we are taking subset containing 10000 samples
data1=data[:10000]

In [ ]:
#Splitting Dataset into Training and Testing with 80% training data and 20% Testing Data
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data1[['question','context']],
                                                    data1['answer'], test_size=0.2, random_state=42)

In [ ]:
#Structure of Train Data
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8000 entries, 9254 to 7270
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  8000 non-null   object
 1   context   8000 non-null   object
dtypes: object(2)
memory usage: 187.5+ KB


In [ ]:
#Splitting the Input "Question" and "Context" to give for Tokenizer as a pair
context_train=x_train['context']
context_test=x_test['context']
x_train=x_train['question']
x_test=x_test['question']

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#Defining T5 Transformer Model and its tokenizer for encoding and decoding operations.
T5Tokenizer = T5Tokenizer.from_pretrained('t5-base')
T5Model = T5ForConditionalGeneration.from_pretrained('t5-base').to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
#Tokenising input "question" and "context" pair into tokens and encoding tokens for both training and testing datasets.
#The returned encoded data is in tensor 2D list format and each each list contains max 512 tokens.
x_train_encoded = T5Tokenizer(x_train.tolist(), context_train.tolist(), padding=True, truncation=True,
                              return_tensors='pt', max_length=512).to(device)
x_test_encoded = T5Tokenizer(x_test.tolist(), context_test.tolist(), padding=True, truncation=True,
                             return_tensors='pt', max_length=512).to(device)

In [ ]:
#Tokenising input "question" and "context" pair into tokens and encoding tokens for both training and testing datasets.
#The returned encoded data is in tensor 2D list format and each each list contains max 512 tokens.
x_train_encoded = T5Tokenizer(x_train.tolist(), context_train.tolist(), padding=True, truncation=True,
                              return_tensors='pt', max_length=512).to(device)
x_test_encoded = T5Tokenizer(x_test.tolist(), context_test.tolist(), padding=True, truncation=True,
                             return_tensors='pt', max_length=512).to(device)

In [ ]:
#Similarily tokenising and encoding the train label
y_train_encoded = T5Tokenizer(y_train.tolist(),padding=True, truncation=True, return_tensors='pt', max_length=512).to(device)

In [ ]:
#The Encoded data contains the ids of tokens and attention mask which describes the importance of tokens.
y_train_encoded

{'input_ids': tensor([[    3, 23143, 14196,  ...,     0,     0,     0],
        [    3, 23143, 14196,  ...,     0,     0,     0],
        [    3, 23143, 14196,  ...,     0,     0,     0],
        ...,
        [    3, 23143, 14196,  ...,     0,     0,     0],
        [    3, 23143, 14196,  ...,     0,     0,     0],
        [    3, 23143, 14196,  ...,     0,     0,     0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')}

In [ ]:
#Transform the input and label pair to a tensor dataset supported by pytorch
Transform_Train_Tensor = TensorDataset(x_train_encoded['input_ids'], x_train_encoded['attention_mask'],
                                       y_train_encoded['input_ids'])
#Randomly Sample the dataset to batches for parallel processing based on given batch size
Sample_Train_Tensor = RandomSampler(Transform_Train_Tensor)
Load_Train_Batches = DataLoader(Transform_Train_Tensor, sampler=Sample_Train_Tensor, batch_size=8)
#Defining Optimizer to update the weights and minimising the loss function
T5ModelOptimizer = torch.optim.AdamW(T5Model.parameters(), lr=2e-5)

In [ ]:
# Defining the training function which does the following:
# Set Epochs to 2
#Iterating through batches
#For each batch computing weights based on input_id, attention_mask and label of that corresponding batch
#computing loss function for the corresponding batch
#computing gradient and updating weights by backpropogating to minimize overall loss function.
T5Model.train()
for i in range(2):
    epoch_loss = 0
    for b in Load_Train_Batches:
        input_id_batch=b[0]
        attention_mask_batch=b[1]
        label_batch = b[2]
        outputs_weights_batch=T5Model(input_ids=input_id_batch,attention_mask=attention_mask_batch,labels=label_batch)
        batch_loss = outputs_weights_batch.loss
        epoch_loss = epoch_loss+batch_loss.item()
        batch_loss.backward()
        T5ModelOptimizer.step()
        T5Model.zero_grad()
        torch.cuda.empty_cache()

    Average_Epoch_Loss = epoch_loss / len(Load_Train_Batches)
    print('Epoch '+str(i)+' Loss = '+str(Average_Epoch_Loss))

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch 0 Loss = 0.4777791198454797
Epoch 1 Loss = 0.07594330419041216


In [ ]:
#Predicting 100 Samples is taking more than 40 GB of GPU RAM which is not feasible with our resources
#So we decided to take subset of test data with 200 samples for predicting the code.
x_test_encoded1=x_test_encoded[:500]

In [ ]:
x_test_encoded1

{'input_ids': tensor([[ 366,  968, 5865,  ...,    0,    0,    0],
         [ 363,   19,    8,  ...,    0,    0,    0],
         [8298,   66,    8,  ...,    0,    0,    0],
         ...,
         [2588,    8, 3056,  ...,    0,    0,    0],
         [ 570, 3056,   13,  ...,    0,    0,    0],
         [ 549,  107,    3,  ...,    0,    0,    0]], device='cuda:0'),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')}

In [ ]:
T5Model.eval()
with torch.no_grad():
    all_outputs = []
    for i in range(500):
        generated_output = T5Model.generate(
            input_ids=x_test_encoded1['input_ids'][i:i+1].to(device),
            attention_mask=x_test_encoded1['attention_mask'][i:i+1].to(device),
            max_length=100,
            num_beams=5,
            early_stopping=True
        )
        all_outputs.append(generated_output)
        torch.cuda.empty_cache()

In [ ]:
predicted_code=[]
for i in all_outputs:
  pred=T5Tokenizer.decode(i[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
  predicted_code.append(pred)
for input_text, code in zip(y_test, predicted_code):
    print(f"Input: {input_text}\nGenerated Code:\n{code}\n")

Input: SELECT assembled FROM table_1827690_4 WHERE elected = "1472"
Generated Code:
SELECT assembled FROM table_1827690_4 WHERE elected = "1472"

Input: SELECT willow_canyon FROM table_11340432_1 WHERE shadow_ridge = "Stallion"
Generated Code:
SELECT willow_canyon FROM table_11340432_1 WHERE shadow_ridge = "Stallion"

Input: SELECT fname, lname FROM authors ORDER BY lname
Generated Code:
SELECT fname, lname FROM authors WHERE alphabetical order of last names

Input: SELECT gvm__kg__technical_capacity FROM table_11497980_1 WHERE model = "15.180E"
Generated Code:
SELECT gvm__kg__technical_capacity FROM table_11497980_1 WHERE model = "15.180e"

Input: SELECT school_club_team FROM table_10015132_16 WHERE years_in_toronto = "1995-96"
Generated Code:
SELECT school_club_team FROM table_10015132_16 WHERE years_in_toronto = 1995-96

Input: SELECT minor_sponsors FROM table_187239_1 WHERE kit_manufacturer = "Olympikus" AND main_sponsor = "Batavo"
Generated Code:
SELECT minor_sponsors FROM table_1

In [ ]:
len(predicted_code)

500

In [ ]:
#Displaying Acutal Test Results and Predicted Test Results.
for test_output, pred_output in zip(y_test[:500], predicted_code):
  print("Test Code Output: "+str(test_output)+"\n"+"Predicted Code Output: "+str(pred_output)+"\n")

Test Code Output: SELECT assembled FROM table_1827690_4 WHERE elected = "1472"
Predicted Code Output: SELECT assembled FROM table_1827690_4 WHERE elected = "1472"

Test Code Output: SELECT willow_canyon FROM table_11340432_1 WHERE shadow_ridge = "Stallion"
Predicted Code Output: SELECT willow_canyon FROM table_11340432_1 WHERE shadow_ridge = "Stallion"

Test Code Output: SELECT fname, lname FROM authors ORDER BY lname
Predicted Code Output: SELECT fname, lname FROM authors WHERE alphabetical order of last names

Test Code Output: SELECT gvm__kg__technical_capacity FROM table_11497980_1 WHERE model = "15.180E"
Predicted Code Output: SELECT gvm__kg__technical_capacity FROM table_11497980_1 WHERE model = "15.180e"

Test Code Output: SELECT school_club_team FROM table_10015132_16 WHERE years_in_toronto = "1995-96"
Predicted Code Output: SELECT school_club_team FROM table_10015132_16 WHERE years_in_toronto = 1995-96

Test Code Output: SELECT minor_sponsors FROM table_187239_1 WHERE kit_manu

In [ ]:
Test_list=y_test[:500].tolist()
Outputlist=predicted_code

In [ ]:
testlist=pd.DataFrame(Test_list)
outputlist=pd.DataFrame(Outputlist)

In [ ]:
pip install rapidfuzz

  Using cached rapidfuzz-3.10.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
Using cached rapidfuzz-3.10.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)


In [ ]:
from rapidfuzz import fuzz
total=0
for i in range(0,500):
  result=fuzz.ratio(testlist[0][i],outputlist[0][i])
  total=total+result

print(total/500)

86.84779392126414


In [ ]:
pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=c55c41aeeed5da9e2081a79e88e19db53f762e6d2e578bbdac636527a29c5b56
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=False)
R1=[]
R2=[]
RL=[]
for i in range(0,500):
  scores = scorer.score(Test_list[i], Outputlist[i])
  R1.append(scores['rouge1'].fmeasure)
  R2.append(scores['rouge2'].fmeasure)
  RL.append(scores['rougeL'].fmeasure)

for i in range(0,10):
  sum_R1=sum(R1)
  sum_R2=sum(R2)
  sum_RL=sum(RL)

print('Rouge1 Score: '+str(sum_R1/500))
print('Rouge2 Score: '+str(sum_R2/500))
print('RougeL Score: '+str(sum_RL/500))

Rouge1 Score: 0.8732782169120589
Rouge2 Score: 0.7864498523807367
RougeL Score: 0.8523645264523053


In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
bleu=[]
for i in range(0,500):
  reference_tokens = [Test_list[i].split()]
  generated_tokens = Outputlist[i].split()
  smoothing = SmoothingFunction().method2
  bleu_score = sentence_bleu(reference_tokens, generated_tokens, smoothing_function=smoothing)
  bleu.append(bleu_score)

sum_bleu=(sum(bleu))/500
print(f"BLEU Score: {sum_bleu:.4f}")

BLEU Score: 0.6295


In [ ]:
#References:
#[1].T5. (n.d.). https://huggingface.co/docs/transformers/en/model_doc/t5
#[2].Papers with Code - Exploring the Limits of Transfer Learning with a Unified Text-to-Text Transformer. (2019, October 23). https://paperswithcode.com/paper/exploring-the-limits-of-transfer-learning#code
#[3].Isaacndirangumuturi. (2023, October 5). Code Generation with T5 Transformer. Kaggle. https://www.kaggle.com/code/isaacndirangumuturi/code-generation-with-t5-transformer
#[4].b-mc2/sql-create-context · Datasets at Hugging Face. (2001, May 12). https://huggingface.co/datasets/b-mc2/sql-create-context?row=0